In [1]:
!pip install vaderSentiment
!pip install transformers==4.23.1
!pip install torch
!pip install functools
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for functools
  Running setup.py clean for functools
Failed to build functools
ERROR: Could not build wh

In [2]:
from google.colab import drive

import pandas as pd
import pickle
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import torch
from torch import nn
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_recall_fscore_support
from torch.optim import AdamW
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch.nn as nn
from functools import partial
from transformers import BertConfig
import accelerate

In [3]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cpu


# Load Data

In [4]:
# Mount Google Drive
drive.mount('/content/gdrive')

# Define the file path
path = "/content/gdrive/MyDrive/Advanced ML/metadata_w_2020articles_cleaned.csv"

# Load data
full_df = pd.read_csv(path)

Mounted at /content/gdrive


In [5]:
# # Create a sample of articles with COVID in the title
# df = full_df[full_df['title'].str.contains('covid', case=False)]

In [6]:
full_df.columns

Index(['Unnamed: 0', 'uuid', 'source', 'year', 'article_text', 'title',
       'title_text'],
      dtype='object')

In [7]:
df = full_df[:100]
df = df.loc[:,['uuid', 'title']]

In [8]:
len(df)

100

In [9]:
df.head(1)

,uuid,title
0,bcbc6bb2-406e-11ee-a96e-33dec8f414a2,"trump shifts tone, says he’s ‘proud’ of fed am..."


# Sentiment Analysis

### Make labels using Vader

In [10]:
analyzer = SentimentIntensityAnalyzer()
sentiment_score = analyzer.polarity_scores('child brutally murdered in los angeles')
sentiment_score

{'neg': 0.677, 'neu': 0.323, 'pos': 0.0, 'compound': -0.8555}

In [11]:
analyzer = SentimentIntensityAnalyzer()
sentiment_score = analyzer.polarity_scores('voters feel disappointed in governments response to COVID')
sentiment_score

{'neg': 0.307, 'neu': 0.693, 'pos': 0.0, 'compound': -0.4767}

In [12]:
analyzer = SentimentIntensityAnalyzer()
sentiment_score = analyzer.polarity_scores('study shows a miraculous 1000 percent improvement in test scores')
sentiment_score

{'neg': 0.0, 'neu': 0.75, 'pos': 0.25, 'compound': 0.4588}

In [13]:
analyzer = SentimentIntensityAnalyzer()
sentiment_score = analyzer.polarity_scores('obama supporters celebrate his historic victory')
sentiment_score

{'neg': 0.0, 'neu': 0.377, 'pos': 0.623, 'compound': 0.765}

In [14]:
analyzer = SentimentIntensityAnalyzer()
sentiment_score = analyzer.polarity_scores('crisis averted as senate passes spending bill')
sentiment_score

{'neg': 0.519, 'neu': 0.481, 'pos': 0.0, 'compound': -0.6597}

Actual headlines

In [15]:
analyzer = SentimentIntensityAnalyzer()
sentiment_score = analyzer.polarity_scores('off the rails: moderators struggle as 10th democratic debate turns into absolute shouting match')
sentiment_score

{'neg': 0.15, 'neu': 0.85, 'pos': 0.0, 'compound': -0.3182}

In [16]:
analyzer = SentimentIntensityAnalyzer()
sentiment_score = analyzer.polarity_scores('netanyahu expresses deep shock over elevator drowning after record rainfall')
sentiment_score

{'neg': 0.224, 'neu': 0.776, 'pos': 0.0, 'compound': -0.3818}

In [17]:
analyzer = SentimentIntensityAnalyzer()
sentiment_score = analyzer.polarity_scores('do you know anybody who trusts the government anymore?: tucker carlson blames official washington for russia collusion hoax',
)
sentiment_score

{'neg': 0.21, 'neu': 0.655, 'pos': 0.135, 'compound': -0.1779}

In [18]:
def classify_sentiment(title):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_score = analyzer.polarity_scores(title)
    if sentiment_score['compound'] >= 0.5:
      return 5
    elif sentiment_score['compound'] >= 0.3:
      return 4
    elif sentiment_score['compound'] <= -0.5:
      return 2
    elif sentiment_score['compound'] <= -0.3:
      return 1
    else:
      return 3

In [19]:
df['sentiment_score'] = df['title'].apply(classify_sentiment)
df = df[df['sentiment_score'].notna()]

In [20]:
# Get counts for each unique value in the 'Category' column
value_counts = df['sentiment_score'].value_counts()

# Create a new DataFrame to represent the table
table_df = pd.DataFrame({'Sentiment Score': value_counts.index, 'Count': value_counts.values})
table_df['Percentage'] = (table_df['Count']/len(df)).round(4)

# Display the resulting table
table_df

,Sentiment Score,Count,Percentage
0,3,46,0.46
1,2,22,0.22
2,1,12,0.12
3,5,11,0.11
4,4,9,0.09


### Make Dataloader (code from Hugging Face)
*   Split into train and test
*   Create data loaders



In [21]:
# df.head(2)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['title'], df['sentiment_score'], test_size=0.2)

In [23]:
# X_train.to_list()

In [24]:
# y_train[:1]

In [ ]:
# Load the BERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# The dataset class
class TheDataset(torch.utils.data.Dataset):

    def __init__(self, texts, sentiments, tokenizer):
        self.texts = texts
        self.sentiments = sentiments
        self.tokenizer  = tokenizer
        self.max_len    = tokenizer.model_max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        sentiments = self.sentiments[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens    = True,
            max_length            = self.max_len,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors        = 'pt',
            padding               = "max_length",
            truncation            = True
        )

        return {
            'input_ids': encoded_text['input_ids'][0],
            'attention_mask': encoded_text['attention_mask'][0],
            'labels': torch.tensor(sentiments, dtype=torch.long) - 1  # Adjust labels to start from 0
        }

# Prepare the Train/Validation sets
train_set_dataset = TheDataset(
    texts    = X_train.tolist(),
    sentiments = y_train.tolist(),
    tokenizer  = tokenizer,
)

valid_set_dataset = TheDataset(
    texts   = X_test.tolist(),
    sentiments = y_test.tolist(),
    tokenizer  = tokenizer,
)

# Load the BERT model
config = BertConfig.from_pretrained("bert-base-uncased")
config.num_labels = 5
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)

# Initialize the bias and weight for the classifier layer
model.classifier.bias.data.fill_(0)
model.classifier.weight.data.normal_(mean=0.0, std=0.02)

# The function to get the accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')  # Change average to 'weighted'
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the training parameters
training_args = TrainingArguments(
    output_dir                  = "./sentiment-analysis",
    num_train_epochs            = 3,
    per_device_train_batch_size = 32,  # Change back to 64 if resources allow
    per_device_eval_batch_size  = 64,
    warmup_steps                = 500,
    weight_decay                = 0.01,
    save_strategy               = "epoch",
    evaluation_strategy         = "steps"
)

# Define trainer
loss_fn = nn.CrossEntropyLoss()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set_dataset,
    eval_dataset=valid_set_dataset,
    compute_metrics=compute_metrics
)

# Start pre-training!
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
# Evaluate the model on the validation set
results = trainer.evaluate() # confirm this is correct

# Print the evaluation results
print("Evaluation Results:", results)

In [ ]:
# Save a fine-tuned model
model_save_directory = "/content/gdrive/MyDrive/model_save_directory/sentiment_bert"
trainer.save_model(model_save_directory)